#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [6]:
import pandas as pd
import pymongo

In [7]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

In [8]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

D:\king9\anaconda3\envs\pythonProject\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000.0,2011-08-17 06:31:31,1145.0,canceled,24,US,1145.0,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000.0,2012-07-23 20:46:48,1851.0,failed,28,US,1851.0,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000.0,2016-07-01 21:55:54,7534.0,successful,254,US,3796.0,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000.0,2017-08-28 15:47:02,16298.0,successful,367,US,2670.0,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000.0,2016-01-25 17:37:10,1.0,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [9]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [10]:
df2 = df_ks.dropna()

### Importer les données

In [11]:
df3 = df2.to_dict(orient='records')

In [12]:
collection.insert_many(df3)

## Question 1  

In [13]:
curse = collection.find().sort([("usd_pledged_real", -1)]).limit(5)
list(curse)

[{'_id': ObjectId('6021db76af3ceee6f7371305'),
  'ID': 342886736,
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2014-08-30',
  'goal': '50000.0',
  'launched': '2014-07-08 10:14:37',
  'pledged': '13285226.36',
  'state': 'successful',
  'backers': '62642',
  'country': 'US',
  'usd pledged': '13285226.36',
  'usd_pledged_real': 13285226.36},
 {'_id': ObjectId('6021db76af3ceee6f7376d2f'),
  'ID': 2103598555,
  'name': 'Pebble 2, Time 2 + All-New Pebble Core',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-30',
  'goal': '1000000.0',
  'launched': '2016-05-24 15:49:52',
  'pledged': '12779843.49',
  'state': 'successful',
  'backers': '66673',
  'country': 'US',
  'usd pledged': '12779843.49',
  'usd_pledged_real': 12779843.49},
 {'_id': ObjectId('6021db77af3ceee6f737a1c6'),
  'ID': 1033978702,
  'name': 'OUY

## Question 2

In [14]:
curse = collection.find({"state":"successful"})
len(list(curse))

52998

## Question 3

In [15]:
curse = collection.aggregate([{"$group" : {"_id" : "$main_category", "sum" : {"$sum" : 1}}}])
list(curse)

[{'_id': 'Film & Video', 'sum': 24982},
 {'_id': 'Food', 'sum': 9805},
 {'_id': 'Dance', 'sum': 1488},
 {'_id': 'Fashion', 'sum': 8943},
 {'_id': 'Art', 'sum': 11260},
 {'_id': 'Crafts', 'sum': 3463},
 {'_id': 'Music', 'sum': 19657},
 {'_id': 'Indie Rock', 'sum': 1},
 {'_id': 'Publishing', 'sum': 15721},
 {'_id': 'Games', 'sum': 13987},
 {'_id': 'Comics', 'sum': 4139},
 {'_id': 'Theater', 'sum': 4305},
 {'_id': 'Journalism', 'sum': 1807},
 {'_id': 'Technology', 'sum': 12871},
 {'_id': 'Photography', 'sum': 4176},
 {'_id': 'Design', 'sum': 11911}]

## Question 4

In [16]:
curse = collection.find( { "$and":[{"country": "FR"}, {"launched": {"$lte" : "2016-01-01"}}] } )
len(list(curse))

330

## Question 5

In [17]:
curse = collection.find( { "$and":[{"country": "US"}, {"goal": {"$gt" : 200000}}] } )
len(list(curse))

1026

## Question 6 

In [18]:
collection.create_index([("name",  "text")])
curse = collection.find( { "$text": { "$search": "Sport" } } )
len(list(curse))

316